In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
class PositionWiseFFN(nn.Module):
    """基于位置的前馈网络"""
    def __init__(self,ffn_num_input,ffn_num_hiddens,ffn_num_outputs,**kwargs):
        super(PositionWiseFFN,self).__init__(**kwargs)
        self.dense1=nn.Linear(ffn_num_input,ffn_num_hiddens)
        self.relu=nn.ReLU()
        self.dense2=nn.Linear(ffn_num_hiddens,ffn_num_outputs)
    def forward(self,x):
        return self.dense2(self.relu(self.dense1(x)))

In [8]:
ffn=PositionWiseFFN(ffn_num_input=4,ffn_num_hiddens=4,ffn_num_outputs=8)
ffn.eval()
ffn(torch.ones((2,3,4))).shape

torch.Size([2, 3, 8])

In [9]:
# 残差连接和层规范化

In [16]:
ln = nn.LayerNorm(2)
bn = nn.BatchNorm1d(2)

In [19]:
x=torch.tensor([[1,2],[2,3]],dtype=torch.float32)
x,x.shape

(tensor([[1., 2.],
         [2., 3.]]),
 torch.Size([2, 2]))

In [17]:
print('layer norm:', ln(X), '\nbatch norm:', bn(X))

layer norm: tensor([[-1.0000,  1.0000],
        [-1.0000,  1.0000]], grad_fn=<NativeLayerNormBackward0>) 
batch norm: tensor([[-1.0000, -1.0000],
        [ 1.0000,  1.0000]], grad_fn=<NativeBatchNormBackward0>)


In [20]:
# 残差连接和层规范化
class AddNorm(nn.Module):
    def __init__(self,normalized_shape,dropout,**kwargs):
        super(AddNorm,self).__init__()
        self.dropout=nn.Dropout(dropout)
        self.ln=nn.LayerNorm(normalized_shape)
        
    def forward(self,X,Y):
        return self.ln(self.dropout(Y) + X)

In [22]:
add_norm = AddNorm([3, 4], 0.5)
add_norm.eval()

AddNorm(
  (dropout): Dropout(p=0.5, inplace=False)
  (ln): LayerNorm((3, 4), eps=1e-05, elementwise_affine=True)
)

In [25]:
add_norm(torch.ones((2, 3, 4)), torch.ones((2, 3, 4))).shape

torch.Size([2, 3, 4])

In [ ]:
class TransformerEncoder(nn.Module):
    def __init__(self,vocab_size,key_size,query_size,value_size,num_hiddens,norm_shape
                 ,ffn_num_input,ffn_num_hiddens,num_heads,num_layers,dropout,use_bias,**kwargs):
        super(TransformerEncoder,self).__init__()
        